In [1]:
import os
os.chdir('..')

In [2]:
from models.bailian import LLM

# export API_KEY='xxx'
llm = LLM(api_key=os.environ.get('API_KEY'))

In [3]:
os.environ['LANG'] = 'ch'
from templates.make_prompts import *

In [4]:
prompt = make_skeleton(
    additional_prompts=[
        make_role('经验丰富的深度学习训练师'),
        make_encourage(),
        make_task(
            task='构建意图识别训练集',
            desc='补充"关于某种疾病的定义"的问题分类，要求问句的文法尽量多变，尽量口语化，构造的句子有长有短，不要出现重复句式。至少列出10条。'
                 '\n语料模板中，疾病名称用[disease]代替，每个句子以`<definition>`开头。\n',
        ),
        make_examples([
            '<definition>什么是[disease]?',
            '<definition>[disease]是什么意思?',
            '<definition>能解释一下[disease]这个病吗?'
        ]),
    ]
)

print(prompt)
print('----')
content = llm(prompt)
print(content)


现在你扮演一名经验丰富的深度学习训练师。请尽你所能去回答问题。
现在需要完成一个"构建意图识别训练集"的任务，这个关于这个任务的详细描述：
补充"关于某种疾病的定义"的问题分类，要求问句的文法尽量多变，尽量口语化，构造的句子有长有短，不要出现重复句式。至少列出10条。
语料模板中，疾病名称用[disease]代替，每个句子以`<definition>`开头。

这是一些样例，样例文本使用`<example>`和`</example>`包裹起来。

<example>
<definition>什么是[disease]?
<definition>[disease]是什么意思?
<definition>能解释一下[disease]这个病吗?
</example>

开始!


----


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


<definition>我听说过[disease]，但具体是啥病呢？
<definition>能否给我科普一下[disease]的定义？
<definition>有人能说说[disease]到底是种什么病吗？
<definition>对于[disease]，它的医学定义是怎样的？
<definition>我想了解一下[disease]是指什么样的健康状况？
<definition>能简单描述一下[disease]的病症特点吗？
<definition>初学者求教，[disease]究竟是怎么界定的？
<definition>能否用通俗的话讲讲[disease]是怎么回事？
<definition>关于[disease]，它在医学上是如何被定义的？
<definition>对于[disease]这种病，它的本质是什么？


In [5]:
prompt = make_skeleton(
    query=make_query('怎么突然就失眠了？'),
    additional_prompts=[
        make_role('医生'),
        make_encourage(),
        make_task(
            task='意图识别',
            desc='病人会询问一些问题，请了解病人问这个问题的目的，识别出病人的潜在意图。'
        ),
        make_access_cls(['疾病定义', '疾病病因', '治疗方法']),
        make_examples([
            ['感冒是什么？', '疾病定义'],
            ['为什么会引起感冒？', '疾病病因'],
            ['感冒要吃什么药？', '治疗方法']
        ]),
    ]
)

print(prompt)
print('----')
content = llm(prompt)
print(content)


现在你扮演一名医生。请尽你所能去回答问题。
现在需要完成一个"意图识别"的任务，这个关于这个任务的详细描述：
病人会询问一些问题，请了解病人问这个问题的目的，识别出病人的潜在意图。
这是一些可选的分类项:
["疾病定义", "疾病病因", "治疗方法"]

这是一些样例，样例文本使用`<example>`和`</example>`包裹起来。

<example>

Question: 感冒是什么？
Answer: 疾病定义

Question: 为什么会引起感冒？
Answer: 疾病病因

Question: 感冒要吃什么药？
Answer: 治疗方法
</example>

开始!

Question: 怎么突然就失眠了？


----
疾病病因
